<a href="https://colab.research.google.com/github/WafaSanaa/Real-time-object-detection-avec-Yolo-X/blob/main/Face_recognition_app_with_encoding_from_cache.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation des librairies python

In [ ]:
pip install opencv-python


In [ ]:
pip install cmake


In [ ]:
pip install dlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user


In [ ]:
pip install tensorflow


In [ ]:
pip install face_recognition

  Using cached dlib-19.24.9.tar.gz (3.4 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for dlib: filename=dlib-19.24.9-cp311-cp311-linux_x86_64.whl size=4539740 sha256=569e612728c9ddf64073dffda6392e8c42b5ad9ebc64f949eebfa5c48eb872cf
  Stored in directory: /root/.cache/pip/wheels/1e/01/c7/2aa7ccf47d6bf0cb20fc98c3817c1fabe8e5f2f8925b3498c9
Successfully built dlib


# Importation des librairies

In [ ]:
import cv2
import numpy as np

import os

In [ ]:
import face_recognition

RuntimeError: Error while calling cudaGetDevice(&the_device_id) in file /tmp/.tmpAmKoPO/sdists-v9/pypi/dlib/19.24.6/JxDrwaAs6QBtviDChgF56/src/dlib/cuda/gpu_data.cpp:204. code: 35, reason: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
path = "C:/Users/rjabj/Desktop/projects/au cour/Face recognition workshop/Projet FACE-RECOGNITION-APP/data"
images = []
classNames = []
personsList = os.listdir(path)
for cl in personsList:
    curPersonn = cv2.imread(f'{path}/{cl}')
    images.append(curPersonn)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

# Création d'encodages des visages

In [ ]:
def findEncodeings(image):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList


In [ ]:
encodeListKnown = findEncodeings(images)
print('Encoding Complete.')




# Démarrer la webcam avec cv2

In [ ]:
cap = cv2.VideoCapture(0)
while True:
    _, img = cap.read()
    cv2.imshow('Face Recogntion', img)
    cv2.waitKey(1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# Démarrer la webcam pour une reconnaissance en temps réel

In [ ]:
cap = cv2.VideoCapture(0)
while True:
    _, img = cap.read()

    imgS = cv2.resize(img, (0,0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    faceCurentFrame = face_recognition.face_locations(imgS)

    encodeCurentFrame = face_recognition.face_encodings(imgS, faceCurentFrame)



    for encodeface, faceLoc in zip(encodeCurentFrame, faceCurentFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeface)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeface)
        #print (faceDis)
        matchIndex = np.argmin(faceDis)


        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            print(name)
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0,0,255), 2)
            cv2.rectangle(img, (x1,y2-35), (x2,y2), (0,0,255), cv2.FILLED)
            cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2)
        else:
            name = "Unknown"

    cv2.imshow('Face Recogntion', img)
    cv2.waitKey(1)

# optionnel : Chargement d'encodages à partir du cache

In [ ]:
import cv2
import numpy as np
import face_recognition
import os
import pickle

# Paths
data_path = r"C:\Users\rjabj\Desktop\projects\au cour\Face recognition workshop\Projet FACE-RECOGNITION-APP\data"
encodings_file = 'encodings.pickle'

# Load known faces & encodings
def load_or_create_encodings(path, enc_file):
    classNames = []
    encodeList = []

    if os.path.exists(enc_file):
        print("Loading encodings from cache...")
        with open(enc_file, 'rb') as f:
            encodeList, classNames = pickle.load(f)
    else:
        print("No cache found. Encoding images...")
        images = []
        personsList = os.listdir(path)

        for cl in personsList:
            img_path = os.path.join(path, cl)
            curImg = cv2.imread(img_path)
            if curImg is not None:
                images.append(curImg)
                classNames.append(os.path.splitext(cl)[0])

        for img in images:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            encodings = face_recognition.face_encodings(img_rgb)
            if encodings:
                encodeList.append(encodings[0])

        # Save to cache
        with open(enc_file, 'wb') as f:
            pickle.dump((encodeList, classNames), f)
        print("Encodings saved to cache.")

    return encodeList, classNames

# Load or compute encodings
encodeListKnown, classNames = load_or_create_encodings(data_path, encodings_file)
print("Encoding complete. Starting recognition...")

# Start webcam
cap = cv2.VideoCapture(1)
frame_count = 0
process_every_n_frames = 5
recognized_faces = []

while True:
    success, frame = cap.read()
    if not success:
        break

    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    if frame_count % process_every_n_frames == 0:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        recognized_faces = []

        for encodeFace, faceLoc in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)

            if faceDis.size > 0:
                matchIndex = np.argmin(faceDis)
                if matches[matchIndex]:
                    name = classNames[matchIndex].upper()
                else:
                    name = "Unknown"
                recognized_faces.append((name, faceLoc))

    for name, faceLoc in recognized_faces:
        y1, x2, y2, x1 = [val * 4 for val in faceLoc]
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.rectangle(frame, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
        cv2.putText(frame, name, (x1 + 6, y2 - 6),
                    cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)

    cv2.imshow('Face Recognition (with Cache)', frame)
    frame_count += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
